## **ResNet - 残差网络**

### **残差块**

假设我们希望学出的理想映射为$f(x)$，作为图5.9上方激活函数的输入。左图虚线框中的部分需要直接拟合出该映射 $f(x)$，而右图虚线框中的部分则需要拟合出有关司徒映射的残差映射$f(x) - x$ . 残差映射在实际中往往更容易优化。

只需将右图虚线框内上方的加权运算（如仿射）的权重和偏差参数学成0，那么 $f(x)$ 即为恒等映射。而实际中，当理想映射 $f(x)$ 极接近于恒等映射时，残差映射也易于捕捉恒等映射的细微波动。

图5.9右图也是ResNet的基础块，即残差块（residual block）。在残差块中，输入可通过跨层的数据线路更快地向前传播。


<img src="ResNet.png"></img>

ResNet没用了VGG 全$3\times 3$卷积层的设计。残差块里首先有2个相同输出通道数的 $3\times 3$卷积层。 每个卷积层后接一个批量归一化层和ReLU激活函数。然后，将输入跳过这2个卷积运算后直接加在最后的ReLU激活函数前。这样的设计要求2个卷积层的输出与输入形状一样，从而可以相加。如果想改变通道数，就需要引入一个额外的 $1\times 1$卷积层来将输入变换成需要的形状后再做相加运算。


In [1]:
import d2lzh as d2l
from mxnet import gluon,init,nd
from mxnet.gluon import nn

class Residual(nn.Block):
    def __init__(self,num_channels,use_1X1conv = False,strides = 1,**kwargs):
        super(Residual, self).__init__(**kwargs)
        self.conv1 = nn.Conv2D(num_channels, kernel_size = 3, padding = 1, strides = strides)
        self.conv2 = nn.Conv2D(num_channels, kernel_size = 3, padding = 1)
        if use_1X1conv:
            self.conv3 = nn.Conv2D(num_channels, kernel_size = 1, strides = strides)
        else:
            self.conv3 = None
        self.bn1 = nn.BatchNorm()
        self.bn2 = nn.BatchNorm()
        
    def forward(self,X):
        Y = nd.relu(self.bn1(self.conv1(X)))
        Y = self.bn2(self.conv2(Y))
        if self.conv3:
            X = self.conv3(X)
        return nd.relu(Y + X)

In [2]:
# 查看输入和输出形状
X = nd.random.uniform(shape=(4, 3, 6, 6))
blk = Residual(6, use_1X1conv=True, strides=2)
blk.initialize()
blk(X).shape

(4, 6, 3, 3)

### **ResNet模型**

ResNet的前两层跟之前介绍的GoogleNet中的一样：在输出通道为64、步幅为2的 $7\times 7$卷积层后接步幅为2的$3\times 3$ 的最大池化层。不同之处在于ResNet每个卷积层后增加的批量归一化层。


In [3]:
net = nn.Sequential()
net.add(nn.Conv2D(64, kernel_size = 7, strides = 2, padding = 3),
        nn.BatchNorm(), nn.Activation("relu"),
        nn.MaxPool2D(pool_size = 3, strides = 2, padding = 1))

GoogLeNet在后面接一4个由Inception块组成的模块。ResNet则使用4个由残差块组成的模块，每个模块使用若干个同样输出通道数的残差块。

第一个模块的通道数同输入通道数一致。由于之前已经使用了步幅为2的最大池化层，所以无须减小高和宽。之后的每个模块在第一个残差块里将上一个模块的通道数翻倍，并将高和宽减半。


In [4]:
def resnet_block(num_channels, num_residuals, first_block = False):
    blk = nn.Sequential()
    for i in range(num_channels):
        if i ==0 and not first_block:
            blk.add(Residual(num_channels,use_1X1conv=True, strides=2))
        else:
            blk.add(Residual(num_channels))
    return blk

In [6]:
# 为ResNet加入所有残差块。这里每个模块使用2个残差块。

net.add(resnet_block(64, 2, first_block=True),
       resnet_block(128,2),
      resnet_block(256,2),
      resnet_block(512,2))

# 加入全局平均池化层后接上全连接层输出
net.add(nn.GlobalAvgPool2D(), nn.Dense(10))

这里，每个模块里有4个卷积（不计算1X1卷积层），加上最开始的卷积层和最后的全连接层，共18层。

In [ ]:
# 训练数据
lr, num_epochs, batch_size, ctx = 0.05, 5, 256, d2l.try_gpu()
net.initialize(force_reinit=True, ctx=ctx, init=init.Xavier())
trainer = gluon.Trainer(net.collect_params(), 'sgd', {'learning_rate': lr})
train_iter, test_iter = d2l.load_data_fashion_mnist(batch_size, resize=96)
d2l.train_ch5(net, train_iter, test_iter, batch_size, trainer, ctx,
              num_epochs)